In [1]:
%load_ext autoreload
%autoreload 2
    
import scprinter as scp
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
import pandas as pd
import numpy as np
import os
import pickle
import torch
import matplotlib as mpl
import h5py
mpl.rcParams['pdf.fonttype'] = 42

In [9]:
work_dir = '/ewsc/rzhang/colitisDSS/tissue'
meta = 'DSS-prim-merge10-ATAC.qc_stats.singlet_epi_only.txt'
frags = 'DSS-prim-merge10-ATAC.reshift.fragments.tsv.gz'
peak = 'DSS-prim-merge10-ATAC.cleanPeaks.fixedwidthpeaks_800bp_300reSized.bed'

In [10]:
meta = pd.read_csv(os.path.join(work_dir, meta), sep='\t')
# meta = meta[meta['P5'] != 'P1.62-2']
meta
batchmapping = {'DSS04':1, 'DSS05':1, 'DSS15':2,  'DSS16':2, 'DSS17':3,  'DSS18':3}
meta['sample_id'] = [f'{xx}_{batchmapping[yy]}' if xx == 'Control' else xx for xx,yy in zip(meta['sample'], meta['batch'])]

In [ ]:
import scprinter as scp
import time
start = time.time()
if os.path.exists(os.path.join(work_dir, 'scprinter.h5ad')):
    printer = scp.load_printer(os.path.join(work_dir, 'scprinter.h5ad'), scp.genome.mm10)
else:
    printer = scp.pp.import_fragments(
                        pathToFrags= os.path.join(work_dir, frags),
                        barcodes=np.unique(meta['barcode']),
                        savename=os.path.join(work_dir, 'scprinter.h5ad'),
                        genome=scp.genome.mm10,
                        min_num_fragments=0, min_tsse=0,
                        sorted_by_barcode=False, 
                        low_memory=False,
                        )
print ("takes", time.time()-start)

# training seq2PRINT

In [24]:
scp.pp.call_peaks(printer, 
                  os.path.join(work_dir, frags),
                  [list(printer.obs_names)], ['all'], preset='seq2PRINT')

INFO  @ Fri, 19 Jul 2024 19:18:06: 
# Command line: callpeak --nomodel -t /data/rzhang/data_15/colitisDSS/tissue/scprinter_supp/all_filtered_frag.tsv.gz --outdir /data/rzhang/data_15/colitisDSS/tissue/scprinter_supp/macs2 -n all -f BEDPE --nolambda --keep-dup all --call-summits --nomodel -B --SPMR --shift 75 --extsize 150 -p 0.01
# ARGUMENTS LIST:
# name = all
# format = BEDPE
# ChIP-seq file = ['/data/rzhang/data_15/colitisDSS/tissue/scprinter_supp/all_filtered_frag.tsv.gz']
# control file = None
# effective genome size = 2.70e+09
# band width = 300
# model fold = [5, 50]
# pvalue cutoff = 1.00e-02
# qvalue will not be calculated and reported as -1 in the final output.
# The maximum gap between significant sites is assigned as the read length/tag size.
# The minimum length of peaks is assigned as the predicted fragment length "d".
# Larger dataset will be scaled towards smaller dataset.
# Range for calculating regional lambda is: 10000 bps
# Broad region calling is off
# Paired-End mo

Reading in peak summit file(s):
NOTE: Assuming all start coordinates are 0-based ..

Padding peak summits by: 500 bp on either side for
Removing peaks overlapping with blacklisted regions and out of bound peaks based on chromosome sizes ..

Filtering overlapping peaks based on peak summit score ..
round: 1 216586 peaks unresolved 215609 peaks selected
round: 2 25 peaks unresolved 25 peaks selected
finish clearing
finish sorting
finished summary


In [25]:
cleaned_peaks = pd.DataFrame(printer.uns["peak_calling"]['all_cleaned'][:])
cleaned_peaks.to_csv(f'{work_dir}/cleaned_narrowPeak.bed', 
                     sep='\t', header=False, index=False)
cleaned_peaks

,0,1,2,3,4,5,6,7
0,chr1,3426703,3427703,all_peak_1,29.8425,29.8425,all,70411
1,chr1,3465776,3466776,all_peak_2,2.1032,2.1032,all,209885
2,chr1,3485023,3486023,all_peak_3,16.8993,16.8993,all,89432
3,chr1,3494744,3495744,all_peak_4a,23.0816,23.0816,all,78357
4,chr1,3495311,3496311,all_peak_4b,8.67919,8.67919,all,117690
...,...,...,...,...,...,...,...,...
215629,chrX,169934326,169935326,all_peak_124842b,12.5533,12.5533,all,101536
215630,chrX,169936775,169937775,all_peak_124843,20.2573,20.2573,all,83628
215631,chrX,169940415,169941415,all_peak_124844a,5.79681,5.79681,all,140852
215632,chrX,169940648,169941648,all_peak_124844b,6.70834,6.70834,all,129998


In [35]:
cell_types = []
bcs = []
for ct in meta['sample_id'].unique():
    cell_types.append(ct)
    bcs.append(list(meta[meta['sample_id'] == ct]['barcode']))

In [47]:
import json
model_configs = []
for fold in range(5):
    for i in range(len(cell_types)):
        ct = cell_types[i]
        model_config= scp.tl.seq_model_config(printer, 
                                         peak_file=f'{work_dir}/cleaned_narrowPeak.bed',
                                         cell_grouping=bcs[i],
                                         group_names=ct,
                                         genome=printer.genome,
                                         fold=fold,
                                         overwrite_bigwig=False,
                                         model_name=f'{ct}',
                                         additional_configs={
                                            "notes": "v3",
                                            "tags": ['colitisDSS',
                                                ct,
                                                f"fold{fold}"]})
        json.dump(model_config, open(f'{work_dir}/{ct}_fold{fold}.JSON', 'w'), indent=4)
        model_configs.append(model_config)

Creating bigwig for Colitis_healing


  0%|          | 0/21 [00:00<?, ?it/s]

Creating bigwig for Control_1


  0%|          | 0/21 [00:00<?, ?it/s]

Creating bigwig for Colitis_recovered


...

Creating bigwig for Colitis_acute


  0%|          | 0/21 [00:00<?, ?it/s]

Creating bigwig for Control_3


  0%|          | 0/21 [00:00<?, ?it/s]

bigwig for Colitis_healing already exists, skip
bigwig for Control_1 already exists, skip
bigwig for Colitis_recovered already exists, skip
bigwig for Control_2 already exists, skip
bigwig for Colitis_acute already exists, skip
bigwig for Control_3 already exists, skip
bigwig for Colitis_healing already exists, skip
bigwig for Control_1 already exists, skip
bigwig for Colitis_recovered already exists, skip
bigwig for Control_2 already exists, skip
bigwig for Colitis_acute already exists, skip
bigwig for Control_3 already exists, skip
bigwig for Colitis_healing already exists, skip
bigwig for Control_1 already exists, skip
bigwig for Colitis_recovered already exists, skip
bigwig for Control_2 already exists, skip
bigwig for Colitis_acute already exists, skip
bigwig for Control_3 already exists, skip
bigwig for Colitis_healing already exists, skip
bigwig for Control_1 already exists, skip
bigwig for Colitis_recovered already exists, skip
bigwig for Control_2 already exists, skip
bigwig f

In [33]:
for ct in cell_types:
    for fold in range(5):
        scp.tl.launch_seq2print(model_config=None,
                                model_config_path=f'{work_dir}/{ct}_fold{fold}.JSON',
                                training_mode='seq2print',
                                temp_dir=f'{work_dir}/temp',
                                model_dir=f'{work_dir}',
                                data_dir='/',
                                enable_wandb=True,
                                gpu_id=0,
                                wandb_project='scPrinter_seq_colitis', silent=True)

CUDA_VISIBLE_DEVICES=0 seq2print_train --config /data/rzhang/data_15/colitisDSS/tissue/Colitis_healing_fold0.JSON --temp_dir /data/rzhang/data_15/colitisDSS/tissue/temp --model_dir /data/rzhang/data_15/colitisDSS/tissue --data_dir / --project scPrinter_seq_clean --enable_wandb
CUDA_VISIBLE_DEVICES=0 seq2print_train --config /data/rzhang/data_15/colitisDSS/tissue/Colitis_healing_fold1.JSON --temp_dir /data/rzhang/data_15/colitisDSS/tissue/temp --model_dir /data/rzhang/data_15/colitisDSS/tissue --data_dir / --project scPrinter_seq_clean --enable_wandb
CUDA_VISIBLE_DEVICES=0 seq2print_train --config /data/rzhang/data_15/colitisDSS/tissue/Colitis_healing_fold2.JSON --temp_dir /data/rzhang/data_15/colitisDSS/tissue/temp --model_dir /data/rzhang/data_15/colitisDSS/tissue --data_dir / --project scPrinter_seq_clean --enable_wandb
CUDA_VISIBLE_DEVICES=0 seq2print_train --config /data/rzhang/data_15/colitisDSS/tissue/Colitis_healing_fold3.JSON --temp_dir /data/rzhang/data_15/colitisDSS/tissue/te

# Calculating seq2PRINT TFBS scores

In [ ]:
# Now fetch models:

import wandb

# Login to your W&B account
wandb.login()

# Set your entity and project
entity = 'ruochiz'  # Replace with your W&B entity (username or team name)
project = 'scPrinter_seq_colitis'  # Replace with your W&B project name

# Initialize the API
api = wandb.Api()

# Get the project
runs = api.runs(f"{entity}/{project}")
condition2model = {}
for run in runs:
    model_name = run.config['savename'] + '-' + run.name + '.pt'
    if run.state == 'finished':
        for tag in run.tags:
            if 'colitisDSS' in tag:
                continue
            if 'fold' in tag:
                continue
            condition = tag
            if condition not in condition2model:
                condition2model[condition] = []
        condition2model[condition].append(model_name)

In [ ]:
groups = list(condition2model.keys())

In [ ]:
for g in groups:
    model_path = condition2model[g]
    scp.tl.seq_tfbs_seq2print(seq_attr_count=None,
                           seq_attr_footprint=None,
                           genome=printer.genome,
                           peak_file=os.path.join(work_dir, peak),
                           save_name=f'{g}_',
                           gpus=[0,1,2,3,4, 5,6,7],
                           overwrite=True,
                           silent=True,
                            model_type='seq2print',
                            model_path=model_path)
for g in groups:
    for m in condition2model[g]:
        model_path = [m]
        scp.tl.seq_tfbs_seq2print(seq_attr_count=None,
                               seq_attr_footprint=None,
                               genome=printer.genome,
                                  group_names=f'{g}_fold{ct}_',
                               region_path=os.path.join(work_dir, peak),
                               gpus=[0,1,2,3,4, 5,6,7],
                                model_type='seq2print',
                                model_path=model_path,
                                 verbose=True,
                              launch=True,
                              return_adata=False, # turn this as True
                              overwrite_seqattr=False,
                              save_key=None, # and input a save_key 
                              save_path=f'{work_dir}')

# Compile TFBS bigwig into motif dataframe

In [37]:
work_dir = '/data/rzhang/data_15/colitisDSS/tissue'
meta = 'DSS-prim-merge10-ATAC.qc_stats.singlet_epi_only.txt'
frags = 'DSS-prim-merge10-ATAC.reshift.fragments.tsv.gz'
peak = 'DSS-prim-merge10-ATAC.cleanPeaks.fixedwidthpeaks_800bp_300reSized.bed'

peak = pd.read_csv(f'{work_dir}/{peak}', sep='\t', header=None)
peak = scp.utils.resize_bed_df(peak, 300)
peak

,0,1,2
0,chr1,3423189,3423489
1,chr1,3427087,3427387
2,chr1,3485535,3485835
3,chr1,3495166,3495466
4,chr1,3514966,3515266
...,...,...,...
127619,chrX,170004337,170004637
127620,chrX,170005168,170005468
127621,chrX,170674756,170675056
127622,chrX,170676444,170676744


In [34]:

# motif match based ov
motifs = scp.motifs.FigR_Mouse_Motifs(,
                           printer.genome, bg='genome'
                           nCores=32,pvalue=5e-5,
                           # mode='motifmatchr'
                          )
motifs.prep_scanner(tf_genes=None,pvalue=5e-5)
res = motifs.scan_motif(np.array(peak), verbose=True)
res = pd.DataFrame(res)
res['chrom'] = res[0]
res['start'] = res[1] + res[7]
res['end'] = res[1] + res[8]
res['TF'] = [xx.split("_")[2] for xx in res[4]]
# loc = loc.drop_duplicates(['chrom','start','end'])
motif_match = res.copy()

  0%|          | 0/356228 [00:00<?, ?it/s]

In [39]:
meta = pd.read_csv(os.path.join(work_dir, meta), sep='\t')
batchmapping = {'DSS04':1, 'DSS05':1, 'DSS15':2,  'DSS16':2, 'DSS17':3,  'DSS18':3}
meta['sample_id'] = [f'{xx}_{batchmapping[yy]}' if xx == 'Control' else xx for xx,yy in zip(meta['sample'], meta['batch'])]

In [41]:
# Load per condition tfbs / per fold tfbs on all cCREs
results = {}
for name_id,name in enumerate(list(meta['sample_id'].unique())):
    import pyBigWig
    from tqdm.auto import *
    for fold in range(5):
        fname = f'{name}_fold{fold}'
        def process_df(df):
            bw = f'/data/rzhang/data_15/colitisDSS/tissue/{fname}_TFBS.bigwig'
            # print (bw)i
            with pyBigWig.open(bw, 'r') as f:
                v_cell = []
                for chr,start,end in zip(tqdm(df[0], disable=True), df[1], df[2]):
                    center = (start + end) // 2
                    # v = f.values(chr, center, center+1, numpy=True)[0]
                    v = np.nan_to_num(f.values(chr, start, end, numpy=True))
                    v_cell.append(v)
            return np.array(v_cell)
        n = 2000  #chunk row size
        list_df = [peak[i:i+n] for i in range(0,peak.shape[0],n)]
        from tqdm.contrib.concurrent import process_map
        v = process_map(process_df, list_df, max_workers=64)
        v = np.concatenate(v, axis=0)
        results[fname] = v
for name_id,name in enumerate(list(meta['sample_id'].unique())):
    import pyBigWig
    from tqdm.auto import *
    fname = f'{name}'
    def process_df(df):
        bw = f'/data/rzhang/data_15/colitisDSS/tissue/{fname}_TFBS.bigwig'
        # print (bw)i
        with pyBigWig.open(bw, 'r') as f:
            v_cell = []
            for chr,start,end in zip(tqdm(df[0], disable=True), df[1], df[2]):
                center = (start + end) // 2
                # v = f.values(chr, center, center+1, numpy=True)[0]
                v = np.nan_to_num(f.values(chr, start, end, numpy=True))
                v_cell.append(v)
        return np.array(v_cell)
    n = 2000  #chunk row size
    list_df = [peak[i:i+n] for i in range(0,peak.shape[0],n)]
    from tqdm.contrib.concurrent import process_map
    v = process_map(process_df, list_df, max_workers=64)
    v = np.concatenate(v, axis=0)
    results[fname] = v

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

...

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

In [42]:
# average within each 10bp tiles
import pyranges
peak.columns = ['Chromosome','Start','End']
temp = pyranges.PyRanges(peak)
temp = temp.window(10)
temp = temp.df
temp.columns = ['chrom','start','end']
temp['index3'] = np.arange(len(temp))
peak.columns = ['chrom','start','end',]
print (v.shape)
for name in results:
    v = results[name]
    res = []
    for i in range(30):
        res.append(v[:, i*10:i*10+10].mean(axis=-1))
    res = np.stack(res, axis=-1)
    res = res.reshape((res.shape[0], -1)).reshape((-1))
    # print (res.shape)
    temp[name] = res

(127624, 300)


In [45]:
temp.to_csv(f'{work_dir}/denovo_cCRE_300bp_tile10bp_TFBS_raw.tsv.gz', sep='\t', index=False)

In [46]:
import bioframe
motif_match2 = motif_match[['chrom','start','end','TF']]
ov = bioframe.overlap(motif_match2, temp)

In [ ]:

TFBS = ov[~ov['chrom_'].isna()]
TFBS

In [49]:
TFBS.to_csv(f'{work_dir}/denovo_cCRE_300bp_motifsites_TFBS_raw.tsv.gz', sep='\t', index=False)

In [ ]:
aggregated = TFBS[['TF'] + list(TFBS.columns[8:])].groupby('TF').mean()
aggregated

In [52]:
aggregated.to_csv(f'{work_dir}/denovo_cCRE_300bp_TFBS_raw_aggregatedbyTF.tsv.gz', sep='\t', index=False)

In [54]:
temp = pd.read_csv(f'{work_dir}/denovo_cCRE_300bp_tile10bp_TFBS_raw.tsv.gz', sep='\t')

In [55]:
mask = np.max(np.array(temp.iloc[:, 4:]), axis=-1) >= 0.2
print (np.sum(mask))
temp = temp.loc[mask].copy()
x1 = np.array(temp[temp.columns[4:]])
from sklearn.preprocessing import quantile_transform
x1 = quantile_transform(x1, axis=0, n_quantiles=100000)
for i, cond1 in enumerate(temp.columns[4:]):
    temp[cond1] = x1[:, i]

895165


In [56]:
temp.to_csv(f'{work_dir}/denovo_cCRE_300bp_tile10bp_TFBS_0.2filt_qtl.tsv.gz', sep='\t', index=False)

In [58]:
import bioframe
motif_match2 = motif_match[['chrom','start','end','TF']]
ov = bioframe.overlap(motif_match2, temp)

In [ ]:

TFBS = ov[~ov['chrom_'].isna()]
TFBS

In [60]:
TFBS.to_csv(f'{work_dir}/denovo_cCRE_300bp_motifsites_TFBS_0.2filt_qtl.tsv.gz', sep='\t', index=False)

In [ ]:
aggregated = TFBS[['TF'] + list(TFBS.columns[8:])].groupby('TF').mean()
aggregated

In [63]:
aggregated.to_csv(f'{work_dir}/denovo_cCRE_300bp_TFBS_0.2filt_qtl_aggregatedbyTF.tsv.gz', sep='\t', index=False)